<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Wage_and_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.4 MB/s eta 0:00:00


In [3]:
def categorize_experience(profile_text):
    """
    פונקציה שממפה קטגוריית ניסיון (EN, MI, SE, EX) על סמך שנות ניסיון מתוכן הטקסט.
    :param profile_text: טקסט פרופיל משתמש (string)
    :return: קטגוריית ניסיון (string)
    """
    import re

    # חיפוש שנות ניסיון בטקסט (למשל: '2 yrs', '10 years', '1 yr')
    experience_matches = re.findall(r"(\d+)\s*(?:yrs?|years?)", profile_text)

    # המרה למספרים של שנות ניסיון
    years_of_experience = sum(map(int, experience_matches))

    # קטגוריות ניסיון לפי מספר שנים
    if years_of_experience < 2:
        return 'EN'  # Entry Level
    elif 2 <= years_of_experience < 5:
        return 'MI'  # Mid Level
    elif 5 <= years_of_experience < 10:
        return 'SE'  # Senior Level
    else:
        return 'EX'  # Executive Level


# קריאה לדוגמה עם טקסט מהפרופיל שסיפקת
profile_text = """
Company       : Next Career
Duration      : 2 yrs 1 mo
--------------------------------------------------
  Job Title    : Career Coach
  Job Duration : 2024 - Present · 1 yr 1 mo
  --------------------------------------------------
  Job Title    : Owner
  Job Duration : 2023 - Present · 2 yrs 1 mo
  --------------------------------------------------
  Job Title     : Global Sourcing Manager
  Job Duration  : 2021 - 2023 · 2 yrs
  --------------------------------------------------
  Job Title     : Director Talent Acquisition
  Job Duration  : 2019 - 2021 · 2 yrs
  --------------------------------------------------
"""

# הפעלת הפונקציה
experience_category = categorize_experience(profile_text)
print(f"Experience Category: {experience_category}")


Experience Category: SE


In [4]:
import pycountry
import re

def extract_job_details_with_iso(job_text):
    """
    חלץ פרטי משרה מתוך טקסט נתון והמר מיקום החברה ל-ISO 3166 Country Code באמצעות pycountry
    :param job_text: טקסט עם פרטי משרה
    :return: מילון עם פרטי המשרה, כולל ISO Country Code
    """

    # חילוץ כותרת המשרה
    job_title = re.search(r"Job Title\s*:\s*(.+)", job_text)
    job_title = job_title.group(1).strip() if job_title else "N/A"

    # חילוץ שם החברה
    company_name = re.search(r"Company\s*:\s*(.+)", job_text)
    company_name = company_name.group(1).strip() if company_name else "N/A"

    # חילוץ מיקום החברה
    company_location = re.search(r"Location\s*:\s*(.+)", job_text)
    company_location = company_location.group(1).strip() if company_location else "N/A"

    # המרת מיקום החברה ל-ISO 3166 Country Code
    country_code = "N/A"
    for country in pycountry.countries:
        if country.name in company_location:
            country_code = country.alpha_2
            break

    # עבודה מהבית: הגדרה לפי תנאים
    if "Remote" in job_text:
        remote_ratio = 100
    elif "Hybrid" in job_text:
        remote_ratio = 50
    elif "On-site" in job_text:
        remote_ratio = 0
    else:
        remote_ratio = "N/A"

    # הרכבת התוצאה
    return {
        "job_title": job_title,
        "company_name": company_name,
        "company_location": company_location,
        "iso_country_code": country_code,
        "remote_ratio": remote_ratio,
        "company_size": "N/A"  # יכול להיות מוגדר על פי מידע נוסף
    }


# קריאה לדוגמה עם הטקסט שסיפקת
job_text = """
Job Title    : Computer Vision Algorithm Engineer
Company      : Camtek
Location     :  Migdal HaEmek, North District, Israel
Description  : Camtek designs, develops, manufactures and markets technologically advanced, cost-effective systems and related software products that are used to enhance processes and yields serving the Advanced Packaging, Memory, CMOS Image Sensors, MEMS, RF, and other segments in the Semiconductors industry.
"""

# הפעלת הפונקציה
job_details = extract_job_details_with_iso(job_text)

# הצגת התוצאה
print(job_details)


{'job_title': 'Computer Vision Algorithm Engineer', 'company_name': 'Camtek', 'company_location': 'Migdal HaEmek, North District, Israel', 'iso_country_code': 'IL', 'remote_ratio': 'N/A', 'company_size': 'N/A'}


In [5]:
import lightgbm as lgb
model = lgb.Booster(model_file='lightgbm_model.txt')
import pandas as pd

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [14]:
new_user = pd.DataFrame({
    'experience_level': [experience_category],         # Senior level
    'company_location': [job_details["iso_country_code"]],         # Company located in the US
    'job_title': [job_details["job_title"]],    # Job title
})



# עמודות קטגוריאליות
categorical_columns = [
    'experience_level', 'employment_type', 'employee_residence',
    'company_location', 'company_size', 'job_title', 'remote_ratio'
]

# המרת עמודות לקטגוריה
for col in categorical_columns:
    if col in new_user.columns:
        new_user[col] = new_user[col].astype('category')

# ווידוא עמודות רלוונטיות לפי X_train
for col in categorical_columns:
    if col not in new_user.columns:
        # הוספת עמודה קטגוריאלית עם ערך ברירת מחדל 'Unknown'
        new_user[col] = "Unknown"
        new_user[col] = new_user[col].astype('category')

# חיזוי שכר
predicted_salary = model.predict(new_user)[0]


Predicted Salary for the new user: $81,885.71


In [17]:
def calculate_salary_range_dynamic(predicted_salary, match_percentage, base_lower_pct=0.9, base_upper_pct=1.0, step_pct=0.05):
    """
    חישוב טווח שכר דינמי המבוסס על רמות שונות של אחוז התאמה, עם ערכים נגזרים באחוזים מהשכר החזוי
    :param predicted_salary: שכר חזוי
    :param match_percentage: אחוז התאמה למשרה
    :param base_lower_pct: אחוז לתחתית הטווח הבסיסי (ברירת מחדל: 90% מהשכר החזוי)
    :param base_upper_pct: אחוז לחלק העליון של הטווח הבסיסי (ברירת מחדל: 100% מהשכר החזוי)
    :param step_pct: אחוז קפיצה לכל רמת התאמה (ברירת מחדל: 5%)
    :return: גבול תחתון ועליון לטווח השכר
    """
    # חישוב טווח בסיסי
    base_lower = predicted_salary * base_lower_pct
    base_upper = predicted_salary * base_upper_pct

    # חישוב גודל הקפיצה באחוזים
    step_size = predicted_salary * step_pct

    # קביעת מספר הצעדים מאחוז התאמה בסיסי (50%)
    steps = (match_percentage - 50) // 10  # כל 10% מייצגים צעד אחד

    # חישוב הטווח החדש
    lower_bound = base_lower + steps * step_size
    upper_bound = base_upper + steps * step_size

    return lower_bound, upper_bound


# דוגמה לשימוש
match_percentage = 65       # אחוז התאמה לדוגמה

# חישוב טווח השכר
lower_salary, upper_salary = calculate_salary_range_dynamic(predicted_salary, match_percentage)

# הצגת התוצאה
print(f"Predicted Salary: ${predicted_salary:,.2f}")
print(f"Salary Range (Match Percentage = {match_percentage}%): ${lower_salary:,.2f} - ${upper_salary:,.2f}")


Predicted Salary: $81,885.71
Salary Range (Match Percentage = 65%): $77,791.42 - $85,979.99
